In [326]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl 
#mpl.use("module://mplcairo.qt") 
from mplcairo import operator_t
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [327]:
def get_bins(df, column, n_bins):
    min_v = df[column].min()
    max_v = df[column].max()
    array = []
    
    for i in range(n_bins+1):
        step = max_v/n_bins
        array.append(int(round(step*i, 0)))
    
    return array

In [328]:
## Get the names and open data
renda_path = ('./data/CENSO/PessoaRenda_PB.csv') 

data = pd.read_csv(renda_path,                  
                sep=';', 
                encoding="utf-8",
                dtype={'Cod_setor': str,
                      'V020': float,
                      'V021': float,
                      'V022': float},
                   na_values = ['X'],
                lineterminator='\n')

In [329]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [330]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [331]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
renda = data[['Cod_setor', 'V020', 'V021', 'V022']]
renda = renda.rename(columns={'V020': 'num_pessoas',
                             'V021': 'num_pessoas_rendimento',
                             'V022': 'rendimento_nominal'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [332]:
populacao = pd.read_csv('./data/POPULACAO/populacao_parse.csv',
                       dtype={'Cod_setor': str})

In [333]:
#populacao.set_index('Cod_setor', inplace=True)
#renda.set_index('Cod_setor', inplace=True)


In [334]:
renda = pd.merge(populacao, renda, on='Cod_setor')

In [335]:
renda['num_pessoas'] = renda['MORADORES']

In [336]:
renda

,Cod_setor,MORADORES,num_pessoas,num_pessoas_rendimento,rendimento_nominal
0,250010605000001,1528,1528,712.0,304733.0
1,250010605000002,500,500,273.0,177732.0
2,250010605000003,889,889,484.0,310937.0
3,250010605000004,860,860,372.0,141471.0
4,250010605000005,667,667,326.0,143194.0
...,...,...,...,...,...
5543,251720905000004,345,345,183.0,81061.0
5544,251720905000005,1105,1105,525.0,212375.0
5545,251740705000001,876,876,469.0,219933.0
5546,251740705000002,603,603,368.0,135068.0


In [337]:
# Set the same inde
renda.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [338]:
setores = setores.join(renda)

In [339]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [340]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [341]:
def get_renda_per_capita(row):
    try:
        renda = row['rendimento_nominal']/row['num_pessoas']
    except (ValueError, ZeroDivisionError):
        print(row['rendimento_nominal'])
        print(row['num_pessoas'])
        return np.nan
    return renda

def get_renda_por_trabalhador(row):
    try:
        renda = row['rendimento_nominal']/row['num_pessoas_rendimento']
    except (ValueError, ZeroDivisionError):
        return np.nan
    return renda

setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)
bairros['renda_per_capita'] = bairros.apply(get_renda_per_capita, axis=1)
bairros['renda_por_trabalhador'] = bairros.apply(get_renda_por_trabalhador, axis=1)

<ipython-input-341-349ba082b907>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
<ipython-input-341-349ba082b907>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)


In [342]:
pessoas_recorte = gpd.read_file('./data/POPULACAO/setores_recorte.shp')

In [343]:
estados = setores_recorte.loc[setores['NM_BAIRRO'] == 'Estados' ]
renda = estados['rendimento_nominal'].sum()

print(renda)
print(bairros.loc['Estados']['rendimento_nominal'])

13819138.0
13819138.0


In [346]:
bairros.loc['Estados'].renda_per_capita

1852.928130866184

In [351]:
#estados['renda_per_capita'].mean()
print(bairros.loc['Estados']['renda_per_capita'])
setores_recorte['renda_per_capita'].describe()

1852.928130866184


count      59.000000
mean     1553.974804
std       839.022851
min       213.003155
25%       870.459411
50%      1579.143713
75%      2104.943126
max      4172.145663
Name: renda_per_capita, dtype: float64

In [286]:
setores_recorte.to_file('./data/RENDA/setores_recorte.shp')
bairros.to_file('./data/RENDA/bairros.shp')

In [287]:
setores_recorte

,geometry,NM_BAIRRO,MORADORES,num_pessoas,num_pessoas_rendimento,rendimento_nominal,renda_per_capita,renda_por_trabalhador
CD_GEOCODI,,,,,,,,
250750705000153,"POLYGON ((-3880337.824 -793550.572, -3880322.8...",Estados,641.0,641.0,449.0,1130863.0,1764.216849,2518.625835
250750705000154,"POLYGON ((-3880206.504 -793549.049, -3880131.9...",Estados,965.0,965.0,613.0,1945252.0,2015.805181,3173.331158
250750705000155,"POLYGON ((-3879433.625 -794026.389, -3879502.9...",Estados,712.0,712.0,468.0,1718193.0,2413.192416,3671.352564
250750705000156,"POLYGON ((-3879858.613 -794140.257, -3879860.2...",Estados,839.0,839.0,475.0,1986198.0,2367.339690,4181.469474
250750705000157,"POLYGON ((-3880720.755 -793909.100, -3880719.7...",Estados,550.0,550.0,359.0,846981.0,1539.965455,2359.278552
...,...,...,...,...,...,...,...,...
250750705000728,"POLYGON ((-3880072.655 -794926.335, -3879957.1...",Expedicionários,805.0,805.0,511.0,1061323.0,1318.413665,2076.953033
250750705000736,"POLYGON ((-3880069.832 -795239.634, -3880070.9...",Torre,611.0,611.0,396.0,657686.0,1076.409165,1660.823232
250750705000742,"POLYGON ((-3879792.753 -793105.640, -3879693.8...",Estados,801.0,801.0,471.0,1478222.0,1845.470662,3138.475584
